###View Data

In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [31]:
train = pd.read_csv("/content/pendapatan_train.csv")
test  = pd.read_csv("/content/pendapatan_test.csv")

# separating string columns to separate list and numerical to separate list

all_data=[train, test]
str_list=[]

for data in all_data:
    for colname, colvalue in data.iteritems(): 
        if type(colvalue[1]) == str:
            str_list.append(colname) 
num_list = data.columns.difference(str_list)
train.head(10)


,id,Umur,Kelas Pekerja,Berat Akhir,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Keuntungan Kapital,Kerugian Capital,Jam per Minggu,Gaji
0,0,40,Wiraswasta,205047,SMA,9,Menikah LDR,Perbaikan Kerajinan,Laki2,0.0,0.0,40.0,>7jt
1,1,33,Pemerintah Lokal,298785,10th,6,Cerai,Supir,Laki2,0.0,0.0,40.0,<=7jt
2,2,26,Wiraswasta,106856,D4,11,Belum Pernah Menikah,Pemuka Agama,Perempuan,0.0,0.0,40.0,<=7jt
3,3,54,Pekerja Bebas Perusahaan,117674,Sarjana,13,Menikah,Sales,Laki2,0.0,0.0,40.0,>7jt
4,4,48,Wiraswasta,268234,SMA,9,Cerai,Perbaikan Kerajinan,Laki2,0.0,0.0,50.0,<=7jt
5,5,27,Pekerja Bebas Perusahaan,376936,SMA,9,Belum Pernah Menikah,Sales,Laki2,0.0,0.0,50.0,<=7jt
6,6,37,Pekerja Bebas Bukan Perusahan,168496,SMA,9,Cerai,Pembersih,Laki2,0.0,0.0,10.0,<=7jt
7,7,51,Wiraswasta,136080,SMA,9,Cerai,Sales,Perempuan,0.0,0.0,31.0,<=7jt
8,8,60,Wiraswasta,232242,SMA,9,Menikah,Sales,Laki2,0.0,0.0,40.0,<=7jt
9,9,44,Pemerintah Lokal,34935,Pendidikan Tinggi,10,Belum Pernah Menikah,Servis Lainnya,Laki2,0.0,0.0,40.0,<=7jt


In [32]:
# Checking null data in entire dataset

train.info()
print('\n\n')
test.info()
# Checking bentuk data vector atau sudah matriks
print('\n\n',test.shape)
print(train.shape)

def missing_value(df):
    miss=[]
    col_list=df.columns
    for i in col_list:
        missing=df[i].isnull().sum()
        miss.append(missing)
        list_of_missing=pd.DataFrame(list(zip(col_list,miss)))
    return list_of_missing

missing_value(test)
missing_value(train)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28794 entries, 0 to 28793
Data columns (total 13 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     28794 non-null  int64  
 1   Umur                   28794 non-null  int64  
 2   Kelas Pekerja          28794 non-null  object 
 3   Berat Akhir            28794 non-null  int64  
 4   Pendidikan             28794 non-null  object 
 5   Jmlh Tahun Pendidikan  28794 non-null  int64  
 6   Status Perkawinan      28794 non-null  object 
 7   Pekerjaan              28794 non-null  object 
 8   Jenis Kelamin          28794 non-null  object 
 9   Keuntungan Kapital     28794 non-null  float64
 10  Kerugian Capital       28794 non-null  float64
 11  Jam per Minggu         28794 non-null  float64
 12  Gaji                   28794 non-null  object 
dtypes: float64(3), int64(4), object(6)
memory usage: 2.9+ MB



<class 'pandas.core.frame.DataFrame'>
RangeInd

,0,1
0,id,0
1,Umur,0
2,Kelas Pekerja,0
3,Berat Akhir,0
4,Pendidikan,0
5,Jmlh Tahun Pendidikan,0
6,Status Perkawinan,0
7,Pekerjaan,0
8,Jenis Kelamin,0
9,Keuntungan Kapital,0


In [33]:
train['Status Perkawinan'].drop_duplicates()

0              Menikah LDR
1                    Cerai
2     Belum Pernah Menikah
3                  Menikah
18                Berpisah
29                   Janda
Name: Status Perkawinan, dtype: object

### Mapping data

In [34]:
# format column Gaji 
train['Gaji']=train['Gaji'].map({'<=7jt': 0, '>7jt': 1})

for dataset in all_data:
    # format Jenis Kelamin
    dataset['Jenis Kelamin'] = dataset['Jenis Kelamin'].map( {'Laki2': 1, 'Perempuan': 0} )

    #format Jenis Status Perkawinan
    dataset['Status Perkawinan'] = dataset['Status Perkawinan'].apply(lambda x: x.strip())
    dataset['Status Perkawinan'] = dataset['Status Perkawinan'].apply(lambda x: 1 if x=='Menikah' else 0)

    #format Jmlh Tahun Pendidikan
    dataset.loc[ dataset['Jmlh Tahun Pendidikan'] <= 6, 'Jmlh Tahun Pendidikan'] 					                                   = 0
    dataset.loc[(dataset['Jmlh Tahun Pendidikan'] > 6) & (dataset['Jmlh Tahun Pendidikan'] <= 9), 'Jmlh Tahun Pendidikan']   = 1
    dataset.loc[(dataset['Jmlh Tahun Pendidikan'] > 9) & (dataset['Jmlh Tahun Pendidikan'] <= 12), 'Jmlh Tahun Pendidikan']  = 2
    dataset.loc[(dataset['Jmlh Tahun Pendidikan'] > 12) & (dataset['Jmlh Tahun Pendidikan'] <= 15), 'Jmlh Tahun Pendidikan'] = 3
    dataset.loc[(dataset['Jmlh Tahun Pendidikan'] > 15) & (dataset['Jmlh Tahun Pendidikan'] <= 16), 'Jmlh Tahun Pendidikan'] = 4

    #format Kelas Pekerja
    kelas_pekerja_mapping = {
        "Tanpa dibayar": 0, "Tidak Pernah Bekerja": 0, "Pekerja Bebas Perusahaan": 2,
        "Pekerja Bebas Bukan Perusahan": 1, "Pemerintah Negara": 4,  "Pemerintah Provinsi": 3,
        'Pemerintah Lokal':3, 'Wiraswasta': 4
    }
    dataset['Kelas Pekerja'] = dataset['Kelas Pekerja'].map(kelas_pekerja_mapping)
    dataset['Kelas Pekerja'] = dataset['Kelas Pekerja'].fillna(0).astype(int)

    #format Umur
    dataset.loc[ dataset['Umur'] <= 18, 'Umur'] 					                 = 0
    dataset.loc[(dataset['Umur'] > 18) & (dataset['Umur'] <= 20), 'Umur']  = 1
    dataset.loc[(dataset['Umur'] > 20) & (dataset['Umur'] <= 22), 'Umur']  = 2
    dataset.loc[(dataset['Umur'] > 22) & (dataset['Umur'] <= 23), 'Umur']  = 3
    dataset.loc[(dataset['Umur'] > 23) & (dataset['Umur'] <= 28), 'Umur']  = 4
    dataset.loc[(dataset['Umur'] > 28) & (dataset['Umur'] <= 35), 'Umur']  = 5
    dataset.loc[(dataset['Umur'] > 35) & (dataset['Umur'] <= 41), 'Umur']  = 6
    dataset.loc[(dataset['Umur'] >= 41),  'Umur']                          = 7

    #format Pekerjaan
    pekerjaan_mapping = {
        "Tech-support": 3, "Spesialis": 3, "Ekesekutif Managerial": 3,
        "Tentara": 1, 'Supir': 1, 'Mesin Inspeksi': 2, 'Perbaikan Kerajinan': 2,
        'Sales': 2, 'Servis Lainnya': 1, "Asisten Rumah Tangga": 0,
        "Petani": 0,'Pembersih':0, 'Pemuka Agama': 1, 'Penjaga': 0
     }
    dataset['Pekerjaan'] = dataset['Pekerjaan'].map(pekerjaan_mapping)
    dataset['Pekerjaan'] = dataset['Pekerjaan'].fillna(0).astype(int)

    #format Jam Per Minggu
    dataset['Jam per Minggu'] = dataset['Jam per Minggu'].fillna(0).astype(int)
    dataset.loc[ dataset['Jam per Minggu'] <= 25, 'Jam per Minggu'] 					                           = 0
    dataset.loc[(dataset['Jam per Minggu'] > 25) & (dataset['Jam per Minggu'] <= 50), 'Jam per Minggu']  = 1
    dataset.loc[(dataset['Jam per Minggu'] > 50) & (dataset['Jam per Minggu'] <= 75), 'Jam per Minggu']  = 2
    dataset.loc[ dataset['Jam per Minggu'] > 75, 'Jam per Minggu']                                       = 3

    #format Pendidikan
    pendidikan_mapping = {
        "SD": 0, "10th": 1, "9th": 1,
        "11th": 1, "7th-8th": 0, "12th": 2,
        "1st-4th": 0, "5th-6th": 0,
        "SMA": 2, "D4": 3, "Sarjana": 4,
        "Pendidikan Tinggi": 4, "D3": 3, "Sekolah Professional": 4,
        "Master": 5, "Doktor": 5
     }
    dataset['Pendidikan'] = dataset['Pendidikan'].map(pendidikan_mapping)
    dataset['Pendidikan'] = dataset['Pendidikan'].fillna(0).astype(int)


# Feature Selection
drop_elements = ['Berat Akhir', 'Keuntungan Kapital', 'Kerugian Capital']
train = train.drop(drop_elements, axis = 1)
test  = test.drop(drop_elements, axis = 1)

train_v = train.values
test_v  = test.values

In [35]:
train.head()

,id,Umur,Kelas Pekerja,Pendidikan,Jmlh Tahun Pendidikan,Status Perkawinan,Pekerjaan,Jenis Kelamin,Jam per Minggu,Gaji
0,0,6,1,2,1,0,2,1,1,1
1,1,5,3,1,0,0,1,1,1,0
2,2,4,1,3,2,0,1,0,1,0
3,3,7,2,4,3,1,2,1,1,1
4,4,7,1,2,1,0,2,1,1,0


# Modeling Data and Creat Submission

In [36]:
from sklearn.model_selection import train_test_split

predictors = train.drop(['Gaji', 'id'], axis=1)
target = train["Gaji"]
x_train, x_val, y_train, y_val = train_test_split(predictors, target, test_size = 0.2, random_state = 0)

In [37]:
# # Gradient Boosting Classifier
# from sklearn.ensemble import GradientBoostingClassifier
# from sklearn.metrics import roc_auc_score

# gbk = GradientBoostingClassifier()
# gbk.fit(x_train, y_train)
# y_pred = gbk.predict(x_val)
# acc_gbk = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_gbk)

In [38]:
# from sklearn.svm import SVC

# svc = SVC()
# svc.fit(x_train, y_train)
# y_pred = svc.predict(x_val)
# acc_svc = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_svc)

In [39]:
# # Random Forest
# from sklearn.ensemble import RandomForestClassifier

# randomforest = RandomForestClassifier()
# randomforest.fit(x_train, y_train)
# y_pred = randomforest.predict(x_val)
# acc_randomforest = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_randomforest)

In [40]:
# # Stochastic Gradient Descent
# from sklearn.linear_model import SGDClassifier

# sgd = SGDClassifier()
# sgd.fit(x_train, y_train)
# y_pred = sgd.predict(x_val)
# acc_sgd = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_sgd)

In [41]:
# # Logistic Regression
# from sklearn.linear_model import LogisticRegression

# logreg = LogisticRegression()
# logreg.fit(x_train, y_train)
# y_pred = logreg.predict(x_val)
# acc_logreg = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_logreg)

In [42]:
# # Perceptron
# from sklearn.linear_model import Perceptron

# perceptron = Perceptron()
# perceptron.fit(x_train, y_train)
# y_pred = perceptron.predict(x_val)
# acc_perceptron = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_perceptron)

In [43]:
# # Linear SVC
# from sklearn.svm import LinearSVC

# linear_svc = LinearSVC()
# linear_svc.fit(x_train, y_train)
# y_pred = linear_svc.predict(x_val)
# acc_linear_svc = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_linear_svc)

In [44]:
# # KNN or k-Nearest Neighbors
# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier()
# knn.fit(x_train, y_train)
# y_pred = knn.predict(x_val)
# acc_knn = round(roc_auc_score(y_val, y_pred) * 100, 2)
# print(acc_knn)

In [45]:
# 1

# Gaussian Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import roc_auc_score

gaussian = GaussianNB()
gaussian.fit(x_train, y_train)
y_pred = gaussian.predict(x_val)
acc_gaussian = round(roc_auc_score(y_val, y_pred) * 100, 2)
print(acc_gaussian)

76.97


In [46]:
#set ids as id and predict Gaji 
ids = test['id']
predictions = gaussian.predict(test.drop('id', axis=1))

#set the output as a dataframe and convert to csv file named submission.csv
output = pd.DataFrame({ 'id' : ids, 'Gaji': predictions })


In [47]:
output

,id,Gaji
0,28794,0
1,28795,0
2,28796,1
3,28797,0
4,28798,0
...,...,...
9594,38388,1
9595,38389,1
9596,38390,0
9597,38391,0


In [48]:
output.to_csv('submission.csv', index=False)